# Выпускной проект по курсу Аналитик данных

# Задание по SQL

# Подготовительные действия

In [1]:
# импортируем библиотеки
from IPython.display import display
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'здесь был логин', # имя пользователя
'pwd': 'здесь был пароль', # пароль
'host': 'здесь был хост',
'port': здесь был порт, # порт подключения
'db': 'здесь было название БД'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Цели исследования

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (так мы исключим из анализа брошюры);
- Определить автора с самой высокой средней оценкой книг (учитывать только книги с 50 и более оценками);
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Исследуем таблицы

Для каждой таблицы выведем первые 5 строк:

### Таблица `books`

In [4]:
# создаём запрос
query = '''
SELECT *
FROM books
LIMIT 5;
'''

In [5]:
# получаем данные на основании запроса
data = pd.io.sql.read_sql(query, con = engine)

In [6]:
# выводим на экран
display(data)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Основная таблица. Содержит все основные сведения о книге - идентификатор книги, название, количество страниц, дату публикации, идентификаторы автора и издателя

### Таблица `authors`

In [7]:
# создаём запрос
query = '''
SELECT *
FROM authors
LIMIT 5;
'''

In [8]:
# получаем данные на основании запроса
data = pd.io.sql.read_sql(query, con = engine)

In [9]:
# выводим на экран
display(data)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица об авторах - содержит идентификатор автора и его ФИО

### Таблица `ratings`

In [10]:
# создаём запрос
query = '''
SELECT *
FROM ratings
LIMIT 5;
'''

In [11]:
# получаем данные на основании запроса
data = pd.io.sql.read_sql(query, con = engine)

In [12]:
# выводим на экран
display(data)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица с оценками читателей, содержит идентификатор книги, идентификатор оценки, оценку и никнейм читателя

### Таблица `reviews`

In [13]:
# создаём запрос
query = '''
SELECT *
FROM reviews
LIMIT 5;
'''

In [14]:
# получаем данные на основании запроса
data = pd.io.sql.read_sql(query, con = engine)

In [15]:
# выводим на экран
display(data)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица со сведениями об обзорах на книги. Содержит идентификатор книги, никнейм обозревателя, идентификатор обзора и текст обзора

### Таблица `publishers`

In [16]:
# создаём запрос
query = '''
SELECT *
FROM publishers
LIMIT 5;
'''

In [17]:
# получаем данные на основании запроса
data = pd.io.sql.read_sql(query, con = engine)

In [18]:
# выводим на экран
display(data)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица со сведениями об издателях. Содержит идентификатор издателя и его название

Изучение баз закончили, можно приступать к решению задач

## Решим поставленные задачи

### Посчитаем, сколько книг вышло после 1 января 2000 года

In [19]:
# создаём запрос
query = '''
SELECT
    COUNT(book_id)
FROM books
WHERE publication_date::DATE > '2000-01-01';
'''

In [20]:
# получаем данные на основании запроса
data = pd.io.sql.read_sql(query, con = engine)

In [21]:
# выводим на экран
display(data)

,count
0,819


**Вывод**

После 01.01.2000 года вышло 819 книг

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [22]:
# создаём запрос
query = '''
SELECT
    b.title,
    (
        SELECT
            COUNT(review_id) AS review_count
        FROM reviews
        WHERE book_id = b.book_id
    ),
    (
        SELECT
            AVG(rating)
        FROM ratings
        WHERE book_id = b.book_id
    ) AS avg_rating
FROM books b, reviews r
GROUP BY b.book_id
order by review_count desc;
'''

In [23]:
# получаем данные на основании запроса
data = pd.io.sql.read_sql(query, con = engine)

In [24]:
# выводим на экран
display(data)

,title,review_count,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


**Вывод**

Для каждой книги посчитано количество обзоров и средний рейтинг

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [25]:
# создаём запрос
query = '''
SELECT
    publishers.publisher,
    COUNT(book_id)
FROM books INNER JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY count DESC
LIMIT 1;
'''

In [26]:
# получаем данные на основании запроса
data = pd.io.sql.read_sql(query, con = engine)

In [27]:
# выводим на экран
display(data)

,publisher,count
0,Penguin Books,42


**Вывод**

Самое продуктивное издательство - `Penguin Books`, оно выпустило 42 книги толще 50 стр.

### Определим автора с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками)

In [28]:
# создаём запрос
query = '''
SELECT
    author,
    AVG(subq.rating) AS avg_rating
FROM (
    SELECT
        author,
        title,
        AVG(rating) AS rating
    FROM (
        SELECT *
        FROM books
        WHERE book_id IN (
            SELECT book_id
            FROM ratings
            GROUP BY book_id
            HAVING COUNT(rating) >= 50
        ) 
    ) AS rated_books
    INNER JOIN authors ON rated_books.author_id = authors.author_id
    INNER JOIN ratings ON rated_books.book_id = ratings.book_id
    GROUP BY author, title
    ORDER BY author
) AS subq
GROUP BY author
ORDER BY avg_rating DESC
LIMIT 1
'''

In [29]:
# получаем данные на основании запроса
data = pd.io.sql.read_sql(query, con = engine)

In [30]:
# выводим на экран
display(data)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


**Вывод**

Дуэт писательницы Джоанны Роулинг и иллюстратора Мэри Гранпрэ имеет наивысший средний рейтинг написанных книг (4.28) среди книг, имеющих 50 и более оценок

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [31]:
# создаём запрос
query = '''
SELECT
    AVG(subq.review_count) AS average_review_count
FROM (
    SELECT COUNT(review_id) AS review_count
        FROM
            reviews
        WHERE username IN (
            SELECT
                username
            FROM ratings
            GROUP BY username
            HAVING COUNT(rating) > 50
            )
    GROUP BY username
    ) AS subq;
'''

In [32]:
# получаем данные на основании запроса
data = pd.io.sql.read_sql(query, con = engine)

In [33]:
# выводим на экран
display(data)

,average_review_count
0,24.333333


Среднее количество обзоров, написанных пользователями, которые поставили более 50 оценок, составляет 24.(3)

## Итог

Результаты исследования занесём в сводную таблицу

|Цель исследования|Результат|
|:-:|:-:|
|Посчитать, сколько книг вышло после 1 января 2000 года|819|
|Для каждой книги посчитать количество обзоров и среднюю оценку|✅|
|Определить издательство, которое выпустило наибольшее число книг толще 50 страниц|Penguin Books 🐧|
|Определить автора с самой высокой средней оценкой книг|J.K. Rowling / Mary GrandPré|
|Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок|24.(3)|

Также необходимо отметить выполнение задач непосредственно проекта:

- [x] Все таблицы изучены;
- [x] все поставленные задачи решены;
- [x] для решения каждой из задач использован ровно 1 SQL-запрос;
- [x] для каждой задачи описан краткий вывод;
- [x] библиотека pandas использована только для хранения и вывода результатов

Мы молодцы 🎉